In [ ]:
#Granger International Market Tests

In [ ]:
#Day 43 - 47

In [1]:
#Granger Causality Test
#https://www.maths.usyd.edu.au/u/jchan/Consult/W10_CompareTwoTimeSeries.pdf
#https://www.machinelearningplus.com/time-series/granger-causality-test-in-python/
from statsmodels.tsa.stattools import grangercausalitytests
#?grangercausalitytests

In [2]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
pd.set_option('display.max_columns', 500)

In [4]:
#Granger Function Variables
maxlag = 1
test = 'ssr_ftest'

In [5]:
#This function goes through all the lags possible
def grangers_causation_matrix(data, variables, test='ssr_ftest', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag = maxlag, verbose=False)
            p_values = [round(test_result[i + 1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [6]:
#This function just gets the single lag data
def grangers_causation_matrix2(data, variables, test='ssr_ftest', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag = [maxlag], verbose=False)
            p_values = test_result[maxlag][0][test][1]
            #p_values = [round(test_result[i + 1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [7]:
#Day 43
d43 = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day43_csi.csv")
d43.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  \
32  20/03/2023 02:06:12            -39                   -3              -36   
33  20/03/2023 02:09:18            -36                   -2              -34   
34  20/03/2023 02:10:19            -46                   -4              -42   
35  20/03/2023 02:13:15            -58                   -3              -55   
36  20/03/2023 02:15:12            -49                    0              -49   

        CSI 300  
32  3947.524414  
33  3944.992920  
34  3944.145508  
35  3943.348633  
36  3941.434814

In [9]:
d43_2 = d43.drop(["Time"], axis = 1)
d43_2.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment      CSI 300
32            -39                   -3              -36  3947.524414
33            -36                   -2              -34  3944.992920
34            -46                   -4              -42  3944.145508
35            -58                   -3              -55  3943.348633
36            -49                    0              -49  3941.434814

In [10]:
#Day 43 Differencing
d43_diff = d43_2.diff()
d43_diff.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0            NaN                  NaN              NaN       NaN
1            2.0                 -4.0              6.0 -4.145263
2          -11.0                  0.0            -11.0 -2.317627
3           21.0                  7.0             14.0 -0.140870
4          -13.0                 -4.0             -9.0  0.405762

In [11]:
#Remove the first row 
d43_diff3 = d43_diff.drop([0])
d43_diff4 = d43_diff3.reset_index(drop = True)
d43_diff4.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0            2.0                 -4.0              6.0 -4.145263
1          -11.0                  0.0            -11.0 -2.317627
2           21.0                  7.0             14.0 -0.140870
3          -13.0                 -4.0             -9.0  0.405762
4           17.0                  5.0             12.0  3.734619

In [15]:
#Test for Stationary 
#https://machinelearningmastery.com/time-series-data-stationary-python/
from statsmodels.tsa.stattools import adfuller
d43_st = d43_diff4["CSI 300"].values
result43 = adfuller(d43_st)
print('ADF Statistic: %f' % result43[0])
print('p-value: %f' % result43[1])
print('Critical Values:')
for key, value in result43[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -4.893193
p-value: 0.000036
Critical Values:
	1%: -3.633
	5%: -2.949
	10%: -2.613


In [24]:
#Granger Function Variables
maxlag = 10
test = 'ssr_ftest'

In [21]:
grangers_causation_matrix2(d43_diff4, variables = d43_diff4.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y               1.000000               0.011328   
Goldstein Sentiment_y         0.555720               1.000000   
Topic Sentiment_y             0.065451               0.065451   
CSI 300_y                     0.505386               0.457317   

                       Topic Sentiment_x  CSI 300_x  
All Sentiment_y                 0.011328   0.322794  
Goldstein Sentiment_y           0.555720   0.286628  
Topic Sentiment_y               1.000000   0.564393  
CSI 300_y                       0.398597   1.000000

In [25]:
grangers_causation_matrix(d43_diff4, variables = d43_diff4.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y                 1.0000                 0.0113   
Goldstein Sentiment_y           0.3661                 1.0000   
Topic Sentiment_y               0.0391                 0.0391   
CSI 300_y                       0.0550                 0.0709   

                       Topic Sentiment_x  CSI 300_x  
All Sentiment_y                   0.0113     0.0798  
Goldstein Sentiment_y             0.3661     0.0890  
Topic Sentiment_y                 1.0000     0.1078  
CSI 300_y                         0.1846     1.0000

In [26]:
#Day 43 
d43_i = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day43_others.csv")
d43_i.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  \
70  20/03/2023 01:52:52            -53                   -3              -50   
71  20/03/2023 01:54:57            -47                   -2              -45   
72  20/03/2023 01:57:46            -31                   -3              -28   
73  20/03/2023 01:59:56            -49                   -6              -43   
74  20/03/2023 02:01:58            -40                   -3              -37   

         Sensex   Nikkei 225   Eur-Usd     Usd-Jpy   Gbp-Usd    Usd-Mxn  \
70  57486.44141  26992.64063  1.067920  131.917999  1.218591  18.947500   
71  57467.17188  26978.60938  1.067920  131.895996  1.218724  18.950920   
72  57418.89844  26975.59961  1.068034  131.824005  1.219081  18.947100   
73  57444.64844  26977.61914  1.068376  131.822998  1.219096  18.941000   
74  57421.53906  26945.66992  1.068262  131.733993  1.218933  18.946449   

    Eur-Gbp  
70  0.87596  
71  0.87604  
72  0.87595  
73  0.87601  
74  0.87601

In [27]:
d43_2i = d43_i.drop(["Time"], axis = 1)
d43_2i.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
70            -53                   -3              -50  57486.44141   
71            -47                   -2              -45  57467.17188   
72            -31                   -3              -28  57418.89844   
73            -49                   -6              -43  57444.64844   
74            -40                   -3              -37  57421.53906   

     Nikkei 225   Eur-Usd     Usd-Jpy   Gbp-Usd    Usd-Mxn  Eur-Gbp  
70  26992.64063  1.067920  131.917999  1.218591  18.947500  0.87596  
71  26978.60938  1.067920  131.895996  1.218724  18.950920  0.87604  
72  26975.59961  1.068034  131.824005  1.219081  18.947100  0.87595  
73  26977.61914  1.068376  131.822998  1.219096  18.941000  0.87601  
74  26945.66992  1.068262  131.733993  1.218933  18.946449  0.87601

In [28]:
#Day 43 Differencing
d43_diffi = d43_2i.diff()
d43_diffi.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment    Sensex  Nikkei 225  \
0            NaN                  NaN              NaN       NaN         NaN   
1          -24.0                 -2.0            -22.0 -69.68359    -2.89063   
2           26.0                  5.0             21.0  53.39062     2.02930   
3          -14.0                 -4.0            -10.0 -97.93750    -3.73828   
4           -3.0                 -6.0              3.0 -53.45312    -2.16016   

    Eur-Usd   Usd-Jpy   Gbp-Usd   Usd-Mxn  Eur-Gbp  
0       NaN       NaN       NaN       NaN      NaN  
1 -0.000228  0.019012  0.000296  0.000198  0.00000  
2  0.000000 -0.018005 -0.000089 -0.002499  0.00011  
3 -0.000114  0.007996  0.000148  0.001600  0.00003  
4  0.000000  0.011002 -0.000089 -0.002232 -0.00004

In [30]:
#Remove the first row 
d43_diff3i = d43_diffi.drop([0])
d43_diff4i = d43_diff3i.reset_index(drop = True)
d43_diff4i.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment     Sensex  Nikkei 225  \
0          -24.0                 -2.0            -22.0  -69.68359    -2.89063   
1           26.0                  5.0             21.0   53.39062     2.02930   
2          -14.0                 -4.0            -10.0  -97.93750    -3.73828   
3           -3.0                 -6.0              3.0  -53.45312    -2.16016   
4           15.0                 19.0             -4.0  116.98046     9.89844   

    Eur-Usd   Usd-Jpy   Gbp-Usd   Usd-Mxn  Eur-Gbp  
0 -0.000228  0.019012  0.000296  0.000198  0.00000  
1  0.000000 -0.018005 -0.000089 -0.002499  0.00011  
2 -0.000114  0.007996  0.000148  0.001600  0.00003  
3  0.000000  0.011002 -0.000089 -0.002232 -0.00004  
4  0.000114  0.014007  0.000044 -0.002850 -0.00020

In [44]:
#Ad Fuller test again on the transformed data
d43_afi = d43_diff4i["Sensex"].values
result39 = adfuller(d43_afi)
print('ADF Statistic: %f' % result43[0])
print('p-value: %f' % result43[1])
print('Critical Values:')
for key, value in result43[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -4.893193
p-value: 0.000036
Critical Values:
	1%: -3.633
	5%: -2.949
	10%: -2.613


In [52]:
#Granger Function Variables
maxlag = 15
test = 'ssr_ftest'

In [51]:
grangers_causation_matrix2(d43_diff4i, variables = d43_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y               1.000000               0.624524   
Goldstein Sentiment_y         0.292480               1.000000   
Topic Sentiment_y             0.482029               0.482029   
Sensex_y                      0.164460               0.569489   
Nikkei 225_y                  0.225606               0.739023   
Eur-Usd_y                     0.734942               0.935758   
Usd-Jpy_y                     0.271988               0.161444   
Gbp-Usd_y                     0.471047               0.668917   
Usd-Mxn_y                     0.630065               0.309518   
Eur-Gbp_y                     0.955241               0.326565   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  Eur-Usd_x  \
All Sentiment_y                 0.624524  0.426959      0.107999   0.915822   
Goldstein Sentiment_y           0.292480  0.184227      0.726886   0.934415   
Topic Sentiment_y               1.000000  0.357703      0.272340   0.711661   
Sensex_y                        0.312471  1.000000      0.312123   0.717855   
Nikkei 225_y                    0.218619  0.608250      1.000000   0.217567   
Eur-Usd_y                       0.684949  0.913291      0.005206   1.000000   
Usd-Jpy_y                       0.485395  0.032120      0.122093   0.936161   
Gbp-Usd_y                       0.214826  0.848669      0.335679   0.344936   
Usd-Mxn_y                       0.333055  0.252289      0.316945   0.106224   
Eur-Gbp_y                       0.879875  0.057493      0.504766   0.805560   

                       Usd-Jpy_x  Gbp-Usd_x  Usd-Mxn_x  Eur-Gbp_x  
All Sentiment_y         0.095867   0.440139   0.275569   0.487612  
Goldstein Sentiment_y   0.216600   0.971183   0.377340   0.489984  
Topic Sentiment_y       0.239931   0.305227   0.151536   0.156485  
Sensex_y                0.147352   0.806372   0.027191   0.491209  
Nikkei 225_y            0.309969   0.515853   0.006696   0.971708  
Eur-Usd_y               0.559291   0.003638   0.138185   0.841588  
Usd-Jpy_y               1.000000   0.884126   0.673358   0.086005  
Gbp-Usd_y               0.833122   1.000000   0.372539   0.992446  
Usd-Mxn_y               0.225601   0.571082   1.000000   0.215940  
Eur-Gbp_y               0.025879   0.288902   0.145287   1.000000

In [53]:
grangers_causation_matrix(d43_diff4i, variables = d43_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y                 1.0000                 0.1438   
Goldstein Sentiment_y           0.2558                 1.0000   
Topic Sentiment_y               0.1270                 0.1270   
Sensex_y                        0.0770                 0.1657   
Nikkei 225_y                    0.1431                 0.0670   
Eur-Usd_y                       0.4300                 0.7630   
Usd-Jpy_y                       0.1538                 0.0596   
Gbp-Usd_y                       0.3076                 0.3678   
Usd-Mxn_y                       0.1370                 0.1061   
Eur-Gbp_y                       0.2554                 0.0352   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  Eur-Usd_x  \
All Sentiment_y                   0.1438    0.1409        0.1080     0.5488   
Goldstein Sentiment_y             0.2558    0.0412        0.3095     0.7118   
Topic Sentiment_y                 1.0000    0.1353        0.2723     0.3097   
Sensex_y                          0.1815    1.0000        0.3121     0.3215   
Nikkei 225_y                      0.1594    0.0608        1.0000     0.0292   
Eur-Usd_y                         0.2167    0.7409        0.0010     1.0000   
Usd-Jpy_y                         0.2467    0.0020        0.0403     0.5366   
Gbp-Usd_y                         0.2148    0.5327        0.1750     0.1823   
Usd-Mxn_y                         0.3331    0.0557        0.0765     0.0562   
Eur-Gbp_y                         0.0748    0.0386        0.4756     0.5790   

                       Usd-Jpy_x  Gbp-Usd_x  Usd-Mxn_x  Eur-Gbp_x  
All Sentiment_y           0.0106     0.1256     0.2756     0.0524  
Goldstein Sentiment_y     0.0583     0.4723     0.1833     0.1415  
Topic Sentiment_y         0.0153     0.1185     0.1010     0.0481  
Sensex_y                  0.0152     0.1300     0.0272     0.1257  
Nikkei 225_y              0.3004     0.2475     0.0025     0.4585  
Eur-Usd_y                 0.2648     0.0034     0.0017     0.3459  
Usd-Jpy_y                 1.0000     0.4469     0.4396     0.0364  
Gbp-Usd_y                 0.1174     1.0000     0.0792     0.7225  
Usd-Mxn_y                 0.1252     0.1486     1.0000     0.1154  
Eur-Gbp_y                 0.0168     0.0049     0.0559     1.0000

In [ ]:
####################################################################################################################

In [54]:
#Day 44
d44 = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day44_csi.csv")
d44.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  \
27  21/03/2023 02:08:28            -35                   -4              -31   
28  21/03/2023 02:10:38             -5                   -5                0   
29  21/03/2023 02:13:41            -24                   -6              -18   
30  21/03/2023 02:16:49            -37                   -5              -32   
31  21/03/2023 02:19:57            -20                   -7              -13   

        CSI 300  
27  3975.508545  
28  3976.130615  
29  3973.332520  
30  3976.227783  
31  3975.337402

In [55]:
d44_2 = d44.drop(["Time"], axis = 1)
d44_2.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment      CSI 300
27            -35                   -4              -31  3975.508545
28             -5                   -5                0  3976.130615
29            -24                   -6              -18  3973.332520
30            -37                   -5              -32  3976.227783
31            -20                   -7              -13  3975.337402

In [56]:
#Day 44 Differencing
d44_diff = d44_2.diff()
d44_diff.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0            NaN                  NaN              NaN       NaN
1           16.0                  0.0             16.0  1.409912
2          -12.0                 -4.0             -8.0  1.415772
3            6.0                  9.0             -3.0 -1.627930
4            1.0                 -4.0              5.0  0.705811

In [57]:
#Remove the first row 
d44_diff3 = d44_diff.drop([0])
d44_diff4 = d44_diff3.reset_index(drop = True)
d44_diff4.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0           16.0                  0.0             16.0  1.409912
1          -12.0                 -4.0             -8.0  1.415772
2            6.0                  9.0             -3.0 -1.627930
3            1.0                 -4.0              5.0  0.705811
4           -9.0                  3.0            -12.0 -0.901123

In [59]:
#Test for Stationary 
#https://machinelearningmastery.com/time-series-data-stationary-python/
from statsmodels.tsa.stattools import adfuller
d44_st = d44["CSI 300"].values
result44 = adfuller(d44_st)
print('ADF Statistic: %f' % result44[0])
print('p-value: %f' % result44[1])
print('Critical Values:')
for key, value in result44[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -5.037740
p-value: 0.000019
Critical Values:
	1%: -3.724
	5%: -2.986
	10%: -2.633


In [67]:
#Granger Function Variables
maxlag = 15
test = 'ssr_ftest'

In [66]:
grangers_causation_matrix2(d44_2, variables = d44_2.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y               1.000000               0.631719   
Goldstein Sentiment_y         0.721105               1.000000   
Topic Sentiment_y             0.459083               0.459083   
CSI 300_y                     0.100553               0.276664   

                       Topic Sentiment_x  CSI 300_x  
All Sentiment_y                 0.631719   0.801394  
Goldstein Sentiment_y           0.721105   0.060645  
Topic Sentiment_y               1.000000   0.309190  
CSI 300_y                       0.494022   1.000000

In [68]:
grangers_causation_matrix(d44_2, variables = d44_2.columns)

ValueError: Insufficient observations. Maximum allowable lag is 9

In [69]:
#Day 44 
d44_i = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day44_others.csv")
d44_i.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  \
49  21/03/2023 01:50:13            -22                   -7              -15   
50  21/03/2023 01:53:20            -40                   -8              -32   
51  21/03/2023 01:55:18            -47                   -3              -44   
52  21/03/2023 01:57:45            -39                   -7              -32   
53  21/03/2023 02:00:07            -36                  -11              -25   

         Sensex   Nikkei 225   Eur-Usd     Usd-Jpy   Gbp-Usd    Usd-Mxn  \
49  57919.17188  26999.78906  1.071582  131.233002  1.225896  18.857550   
50  57945.28125  26981.55078  1.071697  131.242996  1.225986  18.854401   
51  57913.87891  26987.74023  1.071697  131.227997  1.226136  18.853001   
52  57889.73047  26975.59961  1.071697  131.235992  1.226106  18.856100   
53  57910.25000  26945.66992  1.071926  131.220001  1.226091  18.855511   

    Eur-Gbp  
49  0.87380  
50  0.87382  
51  0.87384  
52  0.87388  
53  0.87401

In [70]:
d44_2i = d44_i.drop(["Time"], axis = 1)
d44_2i.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
49            -22                   -7              -15  57919.17188   
50            -40                   -8              -32  57945.28125   
51            -47                   -3              -44  57913.87891   
52            -39                   -7              -32  57889.73047   
53            -36                  -11              -25  57910.25000   

     Nikkei 225   Eur-Usd     Usd-Jpy   Gbp-Usd    Usd-Mxn  Eur-Gbp  
49  26999.78906  1.071582  131.233002  1.225896  18.857550  0.87380  
50  26981.55078  1.071697  131.242996  1.225986  18.854401  0.87382  
51  26987.74023  1.071697  131.227997  1.226136  18.853001  0.87384  
52  26975.59961  1.071697  131.235992  1.226106  18.856100  0.87388  
53  26945.66992  1.071926  131.220001  1.226091  18.855511  0.87401

In [71]:
#Day 44 Differencing
d44_diffi = d44_2i.diff()
d44_diffi.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment     Sensex  Nikkei 225  \
0            NaN                  NaN              NaN        NaN         NaN   
1           -4.0                  4.0             -8.0  -42.23047     0.37109   
2           -1.0                 -4.0              3.0   25.66016    -5.20117   
3           -6.0                  4.0            -10.0   -2.89063     0.37109   
4           20.0                 -1.0             21.0 -106.60937   -14.14062   

    Eur-Usd   Usd-Jpy   Gbp-Usd   Usd-Mxn  Eur-Gbp  
0       NaN       NaN       NaN       NaN      NaN  
1 -0.000115  0.014999 -0.000090  0.000799 -0.00001  
2 -0.000115 -0.024002 -0.000015  0.006001 -0.00003  
3  0.000000 -0.001007 -0.000030 -0.001200 -0.00001  
4  0.000115  0.102005 -0.000601  0.012190  0.00016

In [72]:
#Remove the first row 
d44_diff3i = d44_diffi.drop([0])
d44_diff4i = d44_diff3i.reset_index(drop = True)
d44_diff4i.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment     Sensex  Nikkei 225  \
0           -4.0                  4.0             -8.0  -42.23047     0.37109   
1           -1.0                 -4.0              3.0   25.66016    -5.20117   
2           -6.0                  4.0            -10.0   -2.89063     0.37109   
3           20.0                 -1.0             21.0 -106.60937   -14.14062   
4          -29.0                  0.0            -29.0   19.62109    -5.00977   

    Eur-Usd   Usd-Jpy   Gbp-Usd   Usd-Mxn  Eur-Gbp  
0 -0.000115  0.014999 -0.000090  0.000799 -0.00001  
1 -0.000115 -0.024002 -0.000015  0.006001 -0.00003  
2  0.000000 -0.001007 -0.000030 -0.001200 -0.00001  
3  0.000115  0.102005 -0.000601  0.012190  0.00016  
4 -0.000230  0.007004  0.000255 -0.000420 -0.00005

In [79]:
#Ad Fuller test again on the transformed data
d44_afi = d44_diff4i["Eur-Gbp"].values
result44 = adfuller(d44_afi)
print('ADF Statistic: %f' % result44[0])
print('p-value: %f' % result44[1])
print('Critical Values:')
for key, value in result44[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -6.389976
p-value: 0.000000
Critical Values:
	1%: -3.566
	5%: -2.920
	10%: -2.598


In [86]:
#Granger Function Variables
maxlag = 15
test = 'ssr_ftest'

In [85]:
grangers_causation_matrix2(d44_diff4i, variables = d44_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y               1.000000               0.358700   
Goldstein Sentiment_y         0.538247               1.000000   
Topic Sentiment_y             0.311952               0.311952   
Sensex_y                      0.478644               0.248964   
Nikkei 225_y                  0.708609               0.360446   
Eur-Usd_y                     0.862400               0.820546   
Usd-Jpy_y                     0.715358               0.287464   
Gbp-Usd_y                     0.445954               0.885831   
Usd-Mxn_y                     0.516001               0.244386   
Eur-Gbp_y                     0.392619               0.502691   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  Eur-Usd_x  \
All Sentiment_y                 0.358700  0.478206      0.965077   0.050200   
Goldstein Sentiment_y           0.538247  0.351387      0.151299   0.818083   
Topic Sentiment_y               1.000000  0.134589      0.931008   0.000944   
Sensex_y                        0.613089  1.000000      0.562558   0.737582   
Nikkei 225_y                    0.553229  0.255157      1.000000   0.505268   
Eur-Usd_y                       0.993345  0.410109      0.815919   1.000000   
Usd-Jpy_y                       0.956122  0.159566      0.319604   0.971258   
Gbp-Usd_y                       0.230392  0.524950      0.737514   0.454573   
Usd-Mxn_y                       0.483094  0.880797      0.289504   0.788927   
Eur-Gbp_y                       0.592376  0.293895      0.214289   0.517030   

                       Usd-Jpy_x  Gbp-Usd_x  Usd-Mxn_x  Eur-Gbp_x  
All Sentiment_y         0.956327   0.258151   0.104161   0.754271  
Goldstein Sentiment_y   0.886602   0.631958   0.713335   0.603187  
Topic Sentiment_y       0.965105   0.129952   0.381437   0.431862  
Sensex_y                0.443989   0.880391   0.929567   0.545989  
Nikkei 225_y            0.858969   0.863859   0.888069   0.909167  
Eur-Usd_y               0.049089   0.094389   0.840396   0.745429  
Usd-Jpy_y               1.000000   0.745040   0.099345   0.207590  
Gbp-Usd_y               0.352350   1.000000   0.932909   0.333374  
Usd-Mxn_y               0.967781   0.775917   1.000000   0.243214  
Eur-Gbp_y               0.196827   0.557026   0.876585   1.000000

In [87]:
grangers_causation_matrix(d44_diff4i, variables = d44_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y                 1.0000                 0.0782   
Goldstein Sentiment_y           0.2470                 1.0000   
Topic Sentiment_y               0.1111                 0.1111   
Sensex_y                        0.1979                 0.1373   
Nikkei 225_y                    0.6729                 0.3006   
Eur-Usd_y                       0.4836                 0.0517   
Usd-Jpy_y                       0.0350                 0.1343   
Gbp-Usd_y                       0.0458                 0.4728   
Usd-Mxn_y                       0.0589                 0.0561   
Eur-Gbp_y                       0.0552                 0.0986   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  Eur-Usd_x  \
All Sentiment_y                   0.0782    0.1513        0.2418     0.0248   
Goldstein Sentiment_y             0.2470    0.1438        0.0101     0.6383   
Topic Sentiment_y                 1.0000    0.1346        0.4088     0.0009   
Sensex_y                          0.1455    1.0000        0.4022     0.1428   
Nikkei 225_y                      0.4818    0.0522        1.0000     0.2396   
Eur-Usd_y                         0.6564    0.1657        0.4793     1.0000   
Usd-Jpy_y                         0.0708    0.1251        0.2268     0.4398   
Gbp-Usd_y                         0.0309    0.3556        0.0816     0.1188   
Usd-Mxn_y                         0.1992    0.0133        0.2303     0.0416   
Eur-Gbp_y                         0.0549    0.2939        0.0671     0.0267   

                       Usd-Jpy_x  Gbp-Usd_x  Usd-Mxn_x  Eur-Gbp_x  
All Sentiment_y           0.2320     0.0369     0.0238     0.3372  
Goldstein Sentiment_y     0.1897     0.3935     0.2193     0.3216  
Topic Sentiment_y         0.1266     0.0718     0.0393     0.1045  
Sensex_y                  0.0556     0.0622     0.4578     0.1818  
Nikkei 225_y              0.0585     0.4220     0.3047     0.2142  
Eur-Usd_y                 0.0107     0.0150     0.1158     0.5742  
Usd-Jpy_y                 1.0000     0.1143     0.0161     0.0558  
Gbp-Usd_y                 0.0465     1.0000     0.0420     0.0160  
Usd-Mxn_y                 0.0374     0.1466     1.0000     0.0715  
Eur-Gbp_y                 0.0673     0.4271     0.3677     1.0000

In [ ]:
####################################################################################################################

In [ ]:
####################################################################################################################

In [88]:
#Day 45
d45 = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day45_csi.csv")
d45.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  \
31  22/03/2023 02:08:43             -3                  -15               12   
32  22/03/2023 02:10:48              6                  -12               18   
33  22/03/2023 02:12:46             -9                  -20               11   
34  22/03/2023 02:14:40            -15                  -13               -2   
35  22/03/2023 02:17:36              8                   -6               14   

        CSI 300  
31  3987.351318  
32  3987.664551  
33  3989.350342  
34  3990.265381  
35  3991.643799

In [89]:
d45_2 = d45.drop(["Time"], axis = 1)
d45_2.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment      CSI 300
31             -3                  -15               12  3987.351318
32              6                  -12               18  3987.664551
33             -9                  -20               11  3989.350342
34            -15                  -13               -2  3990.265381
35              8                   -6               14  3991.643799

In [90]:
#Day 45 Differencing
d45_diff = d45_2.diff()
d45_diff.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0            NaN                  NaN              NaN       NaN
1            2.0                  5.0             -3.0 -1.364258
2            6.0                 -4.0             10.0 -0.469238
3          -32.0                  1.0            -33.0 -0.487061
4           16.0                  5.0             11.0  0.945801

In [91]:
#Remove the first row 
d45_diff3 = d45_diff.drop([0])
d45_diff4 = d45_diff3.reset_index(drop = True)
d45_diff4.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0            2.0                  5.0             -3.0 -1.364258
1            6.0                 -4.0             10.0 -0.469238
2          -32.0                  1.0            -33.0 -0.487061
3           16.0                  5.0             11.0  0.945801
4           16.0                  3.0             13.0 -2.106933

In [92]:
#Test for Stationary 
#https://machinelearningmastery.com/time-series-data-stationary-python/
from statsmodels.tsa.stattools import adfuller
d45_st = d45_diff4["CSI 300"].values
result45 = adfuller(d45_st)
print('ADF Statistic: %f' % result45[0])
print('p-value: %f' % result45[1])
print('Critical Values:')
for key, value in result45[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -4.584643
p-value: 0.000138
Critical Values:
	1%: -3.639
	5%: -2.951
	10%: -2.614


In [101]:
#Granger Function Variables
maxlag = 10
test = 'ssr_ftest'

In [98]:
grangers_causation_matrix2(d45_diff4, variables = d45_diff4.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y               1.000000               0.856088   
Goldstein Sentiment_y         0.442204               1.000000   
Topic Sentiment_y             0.770317               0.770317   
CSI 300_y                     0.801940               0.862529   

                       Topic Sentiment_x  CSI 300_x  
All Sentiment_y                 0.856088   0.876210  
Goldstein Sentiment_y           0.442204   0.338899  
Topic Sentiment_y               1.000000   0.743233  
CSI 300_y                       0.707660   1.000000

In [102]:
grangers_causation_matrix(d45_diff4, variables = d45_diff4.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y                 1.0000                 0.0612   
Goldstein Sentiment_y           0.0281                 1.0000   
Topic Sentiment_y               0.2049                 0.2049   
CSI 300_y                       0.4070                 0.0601   

                       Topic Sentiment_x  CSI 300_x  
All Sentiment_y                   0.0612     0.5000  
Goldstein Sentiment_y             0.0281     0.2181  
Topic Sentiment_y                 1.0000     0.3903  
CSI 300_y                         0.4726     1.0000

In [103]:
#Day 45
d45_i = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day45_others.csv")
d45_i.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  Sensex  \
117  NaN            NaN                  NaN              NaN     NaN   
118  NaN            NaN                  NaN              NaN     NaN   
119  NaN            NaN                  NaN              NaN     NaN   
120  NaN            NaN                  NaN              NaN     NaN   
121  NaN            NaN                  NaN              NaN     NaN   

     Nikkei 225  
117         NaN  
118         NaN  
119         NaN  
120         NaN  
121         NaN

In [104]:
d45_2i = d45_i.drop(["Time"], axis = 1)
d45_2i.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment  Sensex  Nikkei 225
117            NaN                  NaN              NaN     NaN         NaN
118            NaN                  NaN              NaN     NaN         NaN
119            NaN                  NaN              NaN     NaN         NaN
120            NaN                  NaN              NaN     NaN         NaN
121            NaN                  NaN              NaN     NaN         NaN

In [105]:
d45_3i = d45_2i.dropna()
d45_3i.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
66          -31.0                -15.0            -16.0  58122.12891   
67            1.0                -11.0             12.0  58128.26953   
68            1.0                 -7.0              8.0  58163.21094   
69           -8.0                -10.0              2.0  58173.76172   
70            9.0                 -9.0             18.0  58197.71094   

     Nikkei 225  
66  27480.08984  
67  27474.84961  
68  27499.81055  
69  27489.83008  
70  27466.60938

In [106]:
d45_3i.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
0           -2.0                -20.0             18.0  58323.69922   
1            6.0                 -5.0             11.0  58335.35156   
2           -4.0                -20.0             16.0  58405.10156   
3          -18.0                -12.0             -6.0  58397.80078   
4            7.0                -17.0             24.0  58337.76172   

    Nikkei 225  
0  27453.56055  
1  27448.83008  
2  27448.82031  
3  27456.50977  
4  27462.00000

In [107]:
#Day 45 Differencing
d45_diffi = d45_3i.diff()
d45_diffi.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment    Sensex  Nikkei 225
0            NaN                  NaN              NaN       NaN         NaN
1            8.0                 15.0             -7.0  11.65234    -4.73047
2          -10.0                -15.0              5.0  69.75000    -0.00977
3          -14.0                  8.0            -22.0  -7.30078     7.68946
4           25.0                 -5.0             30.0 -60.03906     5.49023

In [108]:
#Remove the first row 
d45_diff3i = d45_diffi.drop([0])
d45_diff4i = d45_diff3i.reset_index(drop = True)
d45_diff4i.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment    Sensex  Nikkei 225
0            8.0                 15.0             -7.0  11.65234    -4.73047
1          -10.0                -15.0              5.0  69.75000    -0.00977
2          -14.0                  8.0            -22.0  -7.30078     7.68946
3           25.0                 -5.0             30.0 -60.03906     5.49023
4            2.0                 16.0            -14.0  -7.43359    -3.91016

In [113]:
#Ad Fuller test again on the transformed data
d45_afi = d45_diff4i["Nikkei 225"].values
result45 = adfuller(d45_afi)
print('ADF Statistic: %f' % result45[0])
print('p-value: %f' % result45[1])
print('Critical Values:')
for key, value in result45[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -2.907675
p-value: 0.044463
Critical Values:
	1%: -3.534
	5%: -2.906
	10%: -2.591


In [120]:
#Granger Function Variables
maxlag = 15
test = 'ssr_ftest'

In [119]:
grangers_causation_matrix2(d45_diff4i, variables = d45_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y               1.000000               0.717156   
Goldstein Sentiment_y         0.300714               1.000000   
Topic Sentiment_y             0.516046               0.516046   
Sensex_y                      0.540531               0.584888   
Nikkei 225_y                  0.102549               0.271627   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  
All Sentiment_y                 0.717156  0.647932      0.880407  
Goldstein Sentiment_y           0.300714  0.108977      0.779283  
Topic Sentiment_y               1.000000  0.892466      0.920717  
Sensex_y                        0.669375  1.000000      0.620152  
Nikkei 225_y                    0.361341  0.772721      1.000000

In [121]:
grangers_causation_matrix(d45_diff4i, variables = d45_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y                 1.0000                 0.0658   
Goldstein Sentiment_y           0.1057                 1.0000   
Topic Sentiment_y               0.0130                 0.0130   
Sensex_y                        0.2180                 0.0914   
Nikkei 225_y                    0.0053                 0.0160   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  
All Sentiment_y                   0.0658    0.5022        0.1741  
Goldstein Sentiment_y             0.1057    0.0116        0.3557  
Topic Sentiment_y                 1.0000    0.3578        0.1202  
Sensex_y                          0.1091    1.0000        0.0757  
Nikkei 225_y                      0.0606    0.2463        1.0000

In [ ]:
####################################################################################################################

In [ ]:
####################################################################################################################

In [122]:
#Day 46
d46 = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day46_csi.csv")
d46.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  \
12  23/03/2023 01:36:50            -20                   -1              -19   
13  23/03/2023 01:39:57            -18                   -2              -16   
14  23/03/2023 01:43:17             -4                   -1               -3   
15  23/03/2023 01:45:15            -29                   -3              -26   
16  23/03/2023 01:48:37            -10                   -3               -7   

        CSI 300  
12  4032.530273  
13  4031.811768  
14  4033.173096  
15  4031.544189  
16  4031.387939

In [123]:
d46_2 = d46.drop(["Time"], axis = 1)
d46_2.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment      CSI 300
12            -20                   -1              -19  4032.530273
13            -18                   -2              -16  4031.811768
14             -4                   -1               -3  4033.173096
15            -29                   -3              -26  4031.544189
16            -10                   -3               -7  4031.387939

In [124]:
#Day 46 Differencing
d46_diff = d46_2.diff()
d46_diff.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0            NaN                  NaN              NaN       NaN
1           14.0                  2.0             12.0  2.497803
2            2.0                 -3.0              5.0  0.495117
3           -6.0                 -1.0             -5.0 -0.702881
4          -12.0                  3.0            -15.0  5.936280

In [125]:
#Remove the first row 
d46_diff3 = d46_diff.drop([0])
d46_diff4 = d46_diff3.reset_index(drop = True)
d46_diff4.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0           14.0                  2.0             12.0  2.497803
1            2.0                 -3.0              5.0  0.495117
2           -6.0                 -1.0             -5.0 -0.702881
3          -12.0                  3.0            -15.0  5.936280
4            9.0                 -4.0             13.0  2.146484

In [130]:
#Test for Stationary 
#https://machinelearningmastery.com/time-series-data-stationary-python/
from statsmodels.tsa.stattools import adfuller
d46_st = d46_2["CSI 300"].values
result46 = adfuller(d46_st)
print('ADF Statistic: %f' % result46[0])
print('p-value: %f' % result46[1])
print('Critical Values:')
for key, value in result46[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -1.364293
p-value: 0.599270
Critical Values:
	1%: -3.964
	5%: -3.085
	10%: -2.682


In [101]:
#Granger Function Variables
maxlag = 1
test = 'ssr_ftest'

In [98]:
grangers_causation_matrix2(d46_diff4, variables = d46_diff4.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y               1.000000               0.856088   
Goldstein Sentiment_y         0.442204               1.000000   
Topic Sentiment_y             0.770317               0.770317   
CSI 300_y                     0.801940               0.862529   

                       Topic Sentiment_x  CSI 300_x  
All Sentiment_y                 0.856088   0.876210  
Goldstein Sentiment_y           0.442204   0.338899  
Topic Sentiment_y               1.000000   0.743233  
CSI 300_y                       0.707660   1.000000

In [102]:
grangers_causation_matrix(d46_diff4, variables = d46_diff4.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y                 1.0000                 0.0612   
Goldstein Sentiment_y           0.0281                 1.0000   
Topic Sentiment_y               0.2049                 0.2049   
CSI 300_y                       0.4070                 0.0601   

                       Topic Sentiment_x  CSI 300_x  
All Sentiment_y                   0.0612     0.5000  
Goldstein Sentiment_y             0.0281     0.2181  
Topic Sentiment_y                 1.0000     0.3903  
CSI 300_y                         0.4726     1.0000

In [131]:
#Day 46
d46_i = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day46_others.csv")
d46_i.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  \
38  23/03/2023 01:36:50            -20                   -1              -19   
39  23/03/2023 01:39:57            -18                   -2              -16   
40  23/03/2023 01:43:17             -4                   -1               -3   
41  23/03/2023 01:45:15            -29                   -3              -26   
42  23/03/2023 01:48:37            -10                   -3               -7   

         Sensex   Nikkei 225  
38  58154.14063  27430.28906  
39  58144.92969  27440.15039  
40  58146.62109  27451.91016  
41  58151.17969  27444.41992  
42  58180.64063  27457.90039

In [132]:
d46_2i = d46_i.drop(["Time"], axis = 1)
d46_2i.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
38            -20                   -1              -19  58154.14063   
39            -18                   -2              -16  58144.92969   
40             -4                   -1               -3  58146.62109   
41            -29                   -3              -26  58151.17969   
42            -10                   -3               -7  58180.64063   

     Nikkei 225  
38  27430.28906  
39  27440.15039  
40  27451.91016  
41  27444.41992  
42  27457.90039

In [134]:
d46_3i = d46_2i.dropna()
d46_3i.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
38            -20                   -1              -19  58154.14063   
39            -18                   -2              -16  58144.92969   
40             -4                   -1               -3  58146.62109   
41            -29                   -3              -26  58151.17969   
42            -10                   -3               -7  58180.64063   

     Nikkei 225  
38  27430.28906  
39  27440.15039  
40  27451.91016  
41  27444.41992  
42  27457.90039

In [135]:
d46_3i.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
0            -33                   -7              -26  57960.67969   
1            -28                   -9              -19  57951.53906   
2            -35                   -7              -28  57906.83984   
3            -36                   -6              -30  57955.89844   
4            -31                   -4              -27  57902.03906   

    Nikkei 225  
0  27400.40039  
1  27404.84961  
2  27403.16016  
3  27396.25977  
4  27401.60938

In [136]:
#Day 45 Differencing
d46_diffi = d46_3i.diff()
d46_diffi.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment    Sensex  Nikkei 225
0            NaN                  NaN              NaN       NaN         NaN
1            5.0                 -2.0              7.0  -9.14063     4.44922
2           -7.0                  2.0             -9.0 -44.69922    -1.68945
3           -1.0                  1.0             -2.0  49.05860    -6.90039
4            5.0                  2.0              3.0 -53.85938     5.34961

In [137]:
#Remove the first row 
d46_diff3i = d46_diffi.drop([0])
d46_diff4i = d46_diff3i.reset_index(drop = True)
d46_diff4i.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment    Sensex  Nikkei 225
0            5.0                 -2.0              7.0  -9.14063     4.44922
1           -7.0                  2.0             -9.0 -44.69922    -1.68945
2           -1.0                  1.0             -2.0  49.05860    -6.90039
3            5.0                  2.0              3.0 -53.85938     5.34961
4            0.0                 -3.0              3.0 -38.76953     5.33984

In [142]:
#Ad Fuller test again on the transformed data
d46_afi = d46_diff4i["Nikkei 225"].values
result46 = adfuller(d46_afi)
print('ADF Statistic: %f' % result46[0])
print('p-value: %f' % result46[1])
print('Critical Values:')
for key, value in result46[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -5.092685
p-value: 0.000015
Critical Values:
	1%: -3.601
	5%: -2.935
	10%: -2.606


In [190]:
#Granger Function Variables
maxlag = 12
test = 'ssr_ftest'

In [149]:
grangers_causation_matrix2(d46_diff4i, variables = d46_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y               1.000000               0.226925   
Goldstein Sentiment_y         0.048811               1.000000   
Topic Sentiment_y             0.210116               0.210116   
Sensex_y                      0.777396               0.649828   
Nikkei 225_y                  0.165258               0.851512   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  
All Sentiment_y                 0.226925  0.920264      0.777924  
Goldstein Sentiment_y           0.048811  0.928010      0.966048  
Topic Sentiment_y               1.000000  0.989832      0.699792  
Sensex_y                        0.776033  1.000000      0.059736  
Nikkei 225_y                    0.018987  0.191731      1.000000

In [191]:
grangers_causation_matrix(d46_diff4i, variables = d46_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y                 1.0000                 0.0672   
Goldstein Sentiment_y           0.0174                 1.0000   
Topic Sentiment_y               0.1250                 0.1250   
Sensex_y                        0.0744                 0.0544   
Nikkei 225_y                    0.1153                 0.3163   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  
All Sentiment_y                   0.0672    0.1849        0.2208  
Goldstein Sentiment_y             0.0174    0.1621        0.2952  
Topic Sentiment_y                 1.0000    0.1349        0.4004  
Sensex_y                          0.0186    1.0000        0.0076  
Nikkei 225_y                      0.0190    0.1352        1.0000

In [ ]:
####################################################################################################################

In [ ]:
####################################################################################################################

In [154]:
#Day 47
d47 = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day47_csi.csv")
d47.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  \
25  24/03/2023 02:02:42            -15                    1              -16   
26  24/03/2023 02:04:50             -9                    1              -10   
27  24/03/2023 02:07:12            -17                    1              -18   
28  24/03/2023 02:11:02            -27                   -2              -25   
29  24/03/2023 02:13:41            -11                   -1              -10   

        CSI 300  
25  4019.598633  
26  4020.411621  
27  4021.718018  
28  4025.261230  
29  4025.644531

In [155]:
d47_2 = d47.drop(["Time"], axis = 1)
d47_2.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment      CSI 300
25            -15                    1              -16  4019.598633
26             -9                    1              -10  4020.411621
27            -17                    1              -18  4021.718018
28            -27                   -2              -25  4025.261230
29            -11                   -1              -10  4025.644531

In [156]:
#Day 47 Differencing
d47_diff = d47_2.diff()
d47_diff.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0            NaN                  NaN              NaN       NaN
1           13.0                 -1.0             14.0  0.731690
2           -5.0                  3.0             -8.0  0.594238
3            7.0                 -5.0             12.0 -0.137695
4            6.0                  0.0              6.0 -3.852051

In [157]:
#Remove the first row 
d47_diff3 = d47_diff.drop([0])
d47_diff4 = d47_diff3.reset_index(drop = True)
d47_diff4.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment   CSI 300
0           13.0                 -1.0             14.0  0.731690
1           -5.0                  3.0             -8.0  0.594238
2            7.0                 -5.0             12.0 -0.137695
3            6.0                  0.0              6.0 -3.852051
4          -17.0                  3.0            -20.0  0.563721

In [162]:
#Test for Stationary 
#https://machinelearningmastery.com/time-series-data-stationary-python/
from statsmodels.tsa.stattools import adfuller
d47_st = d47_diff4["CSI 300"].values
result47 = adfuller(d47_st)
print('ADF Statistic: %f' % result47[0])
print('p-value: %f' % result47[1])
print('Critical Values:')
for key, value in result47[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -4.462997
p-value: 0.000229
Critical Values:
	1%: -3.700
	5%: -2.976
	10%: -2.628


In [167]:
#Granger Function Variables
maxlag = 10
test = 'ssr_ftest'

In [168]:
grangers_causation_matrix2(d47_diff4, variables = d47_diff4.columns)

ValueError: Insufficient observations. Maximum allowable lag is 8

In [102]:
grangers_causation_matrix(d47_diff4, variables = d47_diff4.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y                 1.0000                 0.0612   
Goldstein Sentiment_y           0.0281                 1.0000   
Topic Sentiment_y               0.2049                 0.2049   
CSI 300_y                       0.4070                 0.0601   

                       Topic Sentiment_x  CSI 300_x  
All Sentiment_y                   0.0612     0.5000  
Goldstein Sentiment_y             0.0281     0.2181  
Topic Sentiment_y                 1.0000     0.3903  
CSI 300_y                         0.4726     1.0000

In [169]:
#Day 47
d47_i = pd.read_csv("/Users/johnc.burns/Documents/Documents/PhD Year Three/My Paper 5/Granger_Tests/Granger_Int/Day47_others.csv")
d47_i.tail()

Time  All Sentiment  Goldstein Sentiment  Topic Sentiment  \
55  24/03/2023 01:50:53            -11                    0              -11   
56  24/03/2023 01:54:27            -15                    2              -17   
57  24/03/2023 01:56:53            -20                    0              -20   
58  24/03/2023 01:59:17            -25                   -1              -24   
59  24/03/2023 02:02:42            -15                    1              -16   

         Sensex   Nikkei 225  
55  57904.26953  27382.99023  
56  57902.42969  27369.80078  
57  57903.48828  27364.23047  
58  57875.14063  27368.58008  
59  57866.98828  27385.25000

In [170]:
d47_2i = d47_i.drop(["Time"], axis = 1)
d47_2i.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
55            -11                    0              -11  57904.26953   
56            -15                    2              -17  57902.42969   
57            -20                    0              -20  57903.48828   
58            -25                   -1              -24  57875.14063   
59            -15                    1              -16  57866.98828   

     Nikkei 225  
55  27382.99023  
56  27369.80078  
57  27364.23047  
58  27368.58008  
59  27385.25000

In [171]:
d47_3i = d47_2i.dropna()
d47_3i.tail()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
55            -11                    0              -11  57904.26953   
56            -15                    2              -17  57902.42969   
57            -20                    0              -20  57903.48828   
58            -25                   -1              -24  57875.14063   
59            -15                    1              -16  57866.98828   

     Nikkei 225  
55  27382.99023  
56  27369.80078  
57  27364.23047  
58  27368.58008  
59  27385.25000

In [172]:
d47_3i.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment       Sensex  \
0            -38                    0              -38  57997.26172   
1            -34                   -1              -33  58034.98047   
2            -32                   -7              -25  57997.00000   
3            -36                   -2              -34  57911.94922   
4            -30                   -2              -28  57855.57813   

    Nikkei 225  
0  27355.50977  
1  27349.73047  
2  27347.82031  
3  27342.99023  
4  27330.01953

In [173]:
#Day 47 Differencing
d47_diffi = d47_3i.diff()
d47_diffi.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment    Sensex  Nikkei 225
0            NaN                  NaN              NaN       NaN         NaN
1            4.0                 -1.0              5.0  37.71875    -5.77930
2            2.0                 -6.0              8.0 -37.98047    -1.91016
3           -4.0                  5.0             -9.0 -85.05078    -4.83008
4            6.0                  0.0              6.0 -56.37109   -12.97070

In [174]:
#Remove the first row 
d47_diff3i = d47_diffi.drop([0])
d47_diff4i = d47_diff3i.reset_index(drop = True)
d47_diff4i.head()

All Sentiment  Goldstein Sentiment  Topic Sentiment    Sensex  Nikkei 225
0            4.0                 -1.0              5.0  37.71875    -5.77930
1            2.0                 -6.0              8.0 -37.98047    -1.91016
2           -4.0                  5.0             -9.0 -85.05078    -4.83008
3            6.0                  0.0              6.0 -56.37109   -12.97070
4           -2.0                 -3.0              1.0 -51.94922    -6.93945

In [179]:
#Ad Fuller test again on the transformed data
d47_afi = d47_diff4i["Nikkei 225"].values
result47 = adfuller(d47_afi)
print('ADF Statistic: %f' % result47[0])
print('p-value: %f' % result47[1])
print('Critical Values:')
for key, value in result47[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -5.259166
p-value: 0.000007
Critical Values:
	1%: -3.548
	5%: -2.913
	10%: -2.594


In [186]:
#Granger Function Variables
maxlag = 15
test = 'ssr_ftest'

In [185]:
grangers_causation_matrix2(d47_diff4i, variables = d47_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y               1.000000               0.035835   
Goldstein Sentiment_y         0.782448               1.000000   
Topic Sentiment_y             0.058842               0.058842   
Sensex_y                      0.615165               0.015101   
Nikkei 225_y                  0.076244               0.460465   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  
All Sentiment_y                 0.035835  0.815682      0.415748  
Goldstein Sentiment_y           0.782448  0.972383      0.752491  
Topic Sentiment_y               1.000000  0.723210      0.233174  
Sensex_y                        0.619392  1.000000      0.861133  
Nikkei 225_y                    0.147410  0.332254      1.000000

In [187]:
grangers_causation_matrix(d47_diff4i, variables = d47_diff4i.columns)

All Sentiment_x  Goldstein Sentiment_x  \
All Sentiment_y                 1.0000                 0.0074   
Goldstein Sentiment_y           0.0222                 1.0000   
Topic Sentiment_y               0.0212                 0.0212   
Sensex_y                        0.2405                 0.0117   
Nikkei 225_y                    0.0762                 0.2454   

                       Topic Sentiment_x  Sensex_x  Nikkei 225_x  
All Sentiment_y                   0.0074    0.5117        0.1574  
Goldstein Sentiment_y             0.0222    0.1789        0.3800  
Topic Sentiment_y                 1.0000    0.2623        0.0696  
Sensex_y                          0.1873    1.0000        0.1045  
Nikkei 225_y                      0.1421    0.0118        1.0000

In [ ]:
####################################################################################################################